# Cortex Finetuning Experiments

This notebook takes you through evaluating a series of 

In [ ]:
from snowflake.snowpark import Session

connection_params = {
    "account": "...",
    "user": "...",
    "password": "...",
    "role": "...",
    "database": "...",
    "schema": "...",
    "warehouse": "...",
}

# Create a Snowflake session
session = Session.builder.configs(connection_params).create()

In [ ]:
from trulens.core import TruSession

session = TruSession()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)

In [ ]:
from snowflake.cortex import Complete
from trulens.apps.custom import instrument


class Support_Ticket_Classifier:
    @instrument
    def classify_ticket(self, ticket):
        instruction_prompt = """
        You are an agent that helps organize requests that come to our support team. 

        The request category is the reason why the customer reached out. These are the possible types of request categories:

        Roaming fees
        Slow data speed
        Lost phone
        Add new line
        Closing account

        Try doing it for this request and return only the request category only.
        
        """
        rendered_prompt = instruction_prompt + ticket
        label = Complete("mistral-7b", rendered_prompt)
        return label

In [ ]:
stc = Support_Ticket_Classifier()

stc.classify_ticket("I am having trouble with my phone. It is not working properly.")

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_recorder_support_ticket_classifier_mistral_7b = TruCustomApp(
    stc,
    app_name="Support Ticket Classifier",
    app_version="mistral 7b",
)

In [ ]:
with tru_recorder_support_ticket_classifier_mistral_7b:
    stc.classify_ticket("I am having trouble with my phone. It is not working properly.")

In [ ]:
mistral_7b = "mistral-7b"
mistral_large = "mistral-large2"
support_tickets_finetuned_mistral_7b = "SUPPORT_TICKETS_FINETUNED_MISTRAL_7B"
support_ticket_classifier_mistral_7b = Support_Ticket_Classifier(connection_params, session, mistral_7b)
support_ticket_classifier_mistral_large = Support_Ticket_Classifier(connection_params, session, mistral_large)
support_ticket_classifier_mistral_7b_finetuned = Support_Ticket_Classifier(connection_params, session, support_tickets_finetuned_mistral_7b)

In [ ]:
support_tickets = ["I would like to close my account as I am no longer using the services. Please confirm the necessary steps to complete this process. Can you guide me through closing my account? I have found another provider that better suits my needs. I wish to terminate my account due to relocation. Kindly assist me with the required steps.",
    "I am writing to bring to your attention an issue with my recent cell phone bill. During my trip to Europe for two weeks, I noticed additional charges labeled as 'international fees' amounting to $130. These charges were not communicated to me. I request a detailed explanation and a refund. Thank you for addressing this matter promptly.",
    "Hello, I would like to add my daughter to my plan. I need it activated by her birthday at the end of the week.",
    "I am experiencing slow data speeds on my phone. I have attempted to restart my device and check for software updates, but the issue persists. Please provide guidance on resolving this problem. I heavily rely on my phone for work and require a swift solution. Thank you for your support.",
    "I misplaced my phone while using the subway. Despite multiple attempts to call it, it appears to be turned off. I am concerned about my personal data and would like to know the steps for remotely locking and erasing the data on my phone. Please advise on how to proceed. Thank you for your assistance.",
    "My bill is too high after my travel to Canada. I was not informed about additional fees for using my phone abroad. I request a detailed breakdown of these charges and a refund. I appreciate your prompt attention to this issue.",
    "I am moving to france and need to end my plan. Please help me do so by the end of the month.",
    "I am writing to bring to your attention an issue with my recent cell phone bill. During my trip to Europe for two weeks, I noticed additional charges labeled as 'international fees' amounting to $130. These charges were not communicated to me. I request a detailed explanation and a refund. Thank you for addressing this matter promptly.",
    "Hello, I would like to add a new line to my existing cell phone plan. Kindly activate it within the next 9 days. If there are any further steps or information needed, please inform me. Thank you for your prompt assistance.",
    "I am experiencing slow data speeds on my phone. I have attempted to restart my device and check for software updates, but the issue persists. Please provide guidance on resolving this problem. I heavily rely on my phone for work and require a swift solution. Thank you for your support.",
]

In [ ]:
from trulens.core import Feedback
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on_input_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_recorder_support_ticket_classifier_mistral_7b = TruCustomApp(
    support_ticket_classifier_mistral_7b,
    app_name="Support Ticket Classifier",
    app_version="mistral 7b",
)

In [ ]:
tru_recorder_support_ticket_classifier_mistral_large = TruCustomApp(
    support_ticket_classifier_mistral_large,
    app_name="Support Ticket Classifier",
    app_version="mistral large",
    metadata={"model": "llama3.1-405b"},
)

In [ ]:
tru_recorder_support_ticket_classifier_mistral_7b_finetuned = TruCustomApp(
    support_ticket_classifier_mistral_7b_finetuned,
    app_name="Support Ticket Classifier",
    app_version="mistral 7b finetuned",
    metadata={"model": "mistral-7b finetuned"},
)

In [ ]:
for ticket in support_tickets:
    print(f"Ticket: {ticket}")
    with tru_recorder_support_ticket_classifier_mistral_7b as recording:
        label_small = support_ticket_classifier_mistral_7b.classify_ticket(ticket)
        print(f"mistral 7b label: {label_small}")

In [ ]:
for ticket in support_tickets:
    print(f"Ticket: {ticket}")
    with tru_recorder_support_ticket_classifier_mistral_large:
        label_large = support_ticket_classifier_mistral_large.classify_ticket(ticket)
        print(f"mistral large label: {label_large}")

In [ ]:
for ticket in support_tickets:
    print(f"Ticket: {ticket}")
    with tru_recorder_support_ticket_classifier_mistral_7b_finetuned:
        label_finetuned = support_ticket_classifier_mistral_7b_finetuned.classify_ticket(ticket)
        print(f"mistral 7b finetuned label: {label_finetuned}")